# REVIEW SUMMARIZER
## TRIPADVISOR: HOTELS

*   Esteban Ariza
*   Johan Giraldo
*   Mateo Valdes

## Prerequisites

In [ ]:
#%pip install transformers
#%pip install torch
#%pip install sentencepiece
#%pip install rouge-score
#%pip install evaluate

In [ ]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import pandas as pd
import csv
from rouge_score import rouge_scorer, scoring
import evaluate

In [ ]:
COLUMNS_NAME = ['ORIGINAL_TEXT', 'SUMMARIZED_TEXT']

In [ ]:
try:
    writer = csv.DictWriter(open('summarized_reviews.csv', 'w', encoding='UTF8', newline=''), fieldnames=COLUMNS_NAME, delimiter=',', lineterminator='\r')
    writer.writeheader()
except IOError:
    print("I/O error")

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def summarize(review):
    tokenized_text = tokenizer.encode('summarize: ' + review, return_tensors="pt").to(device)
    summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
    row = {}
    row[COLUMNS_NAME[0]] = review
    row[COLUMNS_NAME[1]] = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    try:
        writer.writerow(row)
    except IOError:
                print("I/O error")
    print('Summarized: ' + row[COLUMNS_NAME[0]] + ' to: ' + row[COLUMNS_NAME[1]])

In [ ]:
data = pd.read_csv('tripadvisor_hotels_sustainables_v2.csv')
#data['REVIEW_TEXT'].apply(summarize)

In [ ]:
rouge = evaluate.load('rouge')
predictions_df = pd.read_csv('summarized_reviews.csv')
predictions = predictions_df[COLUMNS_NAME[1]].tolist()
references = predictions_df[COLUMNS_NAME[0]].tolist()
results = rouge.compute(predictions=predictions, references=references)
print(results)